In [1]:
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

<h1>Moments</h1>

In [2]:
src = cv.imread('/Users/kolsha/Pictures/moments/multiple-blob.png')
src_gray = cv.cvtColor(src, cv.COLOR_BGR2GRAY)
#src_gray = cv.medianBlur(src_gray, 5)
src = cv.cvtColor(src, cv.COLOR_BGR2RGB)





@interact(canny=(20.0, 255.0), image=(1,6), rotate=(0, 360), scale=(0.5, 1.5))
def moments(canny, image, rotate, scale):
    
    tpl = cv.imread('/Users/kolsha/Pictures/moments/'+ str(image) + '.jpg')
    #tpl = cv.resize(tpl, (0,0), fx=0.5, fy=0.5)
    
    num_rows, num_cols = tpl.shape[:2]

    rotation_matrix = cv.getRotationMatrix2D((num_cols/2, num_rows/2), rotate, scale)
    tpl = cv.warpAffine(tpl, rotation_matrix, (num_cols, num_rows))
    
    tpl_gray = cv.cvtColor(tpl, cv.COLOR_BGR2GRAY)
    tpl = cv.cvtColor(tpl, cv.COLOR_BGR2RGB)
    
    src_canny = cv.Canny(src_gray, canny, 250, apertureSize=3)  # binI
    tpl_canny = cv.Canny(tpl_gray, canny, 250, apertureSize=3)  # binT
    
    dst_src = np.copy(src)
    dst_tpl = np.copy(tpl)

    # находим контуры изображения
    _, contoursI, hierarchyI = cv.findContours(src_canny, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)

    # находим контуры шаблона
    _, contoursT, hierarchyT = cv.findContours(tpl_canny, cv.RETR_LIST, cv.CHAIN_APPROX_SIMPLE)

    seqT = 0
    perimT = 0
    # находим самый длинный контур
    for seq0 in range(len(contoursT)):
        perim = len(contoursT[seq0])
        if perim > perimT:
            perimT = perim
            seqT = seq0
    
    color = (np.random.randint(255), np.random.randint(255), np.random.randint(255))
    cv.drawContours(dst_tpl, contoursT, seqT, color, 2, cv.LINE_8, hierarchyT, 0)


    seqM = 0
    matchM = 100.0

    # обходим контуры изображения

    for seq0 in range(len(contoursI)):
        match0 = cv.matchShapes(contoursI[seq0], contoursT[seqT], cv.CONTOURS_MATCH_I3, 0.0)
        if match0 <= matchM:
            matchM = match0
            seqM = seq0
    
    cv.drawContours(dst_src, contoursI, seqM, color, 2, cv.LINE_8, hierarchyI, 0)
    
    titles = ['C Image', 'C Tpl']#['Original Image', 'Tpl', 'C Image', 'C Tpl']
    images = [dst_src, dst_tpl]#[src, tpl, dst_src, dst_tpl]
    fig = plt.gcf()
    fig.set_size_inches(20, 15.5)
    for i in range(len(images)):
        plt.subplot(2, 3, i+1),
        plt.imshow(images[i])
        plt.title(titles[i])
        plt.xticks([]),
        plt.yticks([])
    plt.show()
    return (canny)
    
    
    
    

interactive(children=(FloatSlider(value=137.5, description='canny', max=255.0, min=20.0), IntSlider(value=3, d…